<a href="https://colab.research.google.com/github/yasyamauchi/education/blob/main/decision_tree_BME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 決定木 (けっていぎ，decision tree)  
  
[Google Machine Learning: 上級コース: デシジョンフォレスト](https://developers.google.com/machine-learning/decision-forests/practice?hl=ja)より  

TF-DF（TensorFlow デシジョン フォレスト）ライブラリを使用する．

## 準備  
  
  メッセージがたくさん出るが気にしない．

In [ ]:
!pip install tensorflow_decision_forests

In [ ]:
import numpy as np
import pandas as pd
import tensorflow_decision_forests as tfdf

## データの読み込み  
  
  この例では"Palmer Penguins"データセットを使用する．  
### データの中身  
3種類のペンギン(合計344羽)  
* チンストラップ(Chinstrap)
* ジェンツー(Gentoo)
* アデリー(Adelie)  
  
の，次のデータを使用する．  
* 生息地 (island)
* くちばしの長さ (bill_length)
* くちばしの上下幅 (bill_depth)
* 羽の長さ (flipper_length)
* 体重 (body_mass)
* 性別 (sex)
* 誕生年？ (year)  
  
最初の3羽のデータを表示する．

In [ ]:
path = "https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins.csv"
pandas_dataset = pd.read_csv(path)

# Display the first 3 examples.
pandas_dataset.head(3)

ラベルを確認する．文字列のままではわかりづらいので，ペンギンの種については整数ラベルに変換する．

In [ ]:
label = "species"

classes = list(pandas_dataset[label].unique())
print(f"Label classes: {classes}")
# >> Label classes: ['Adelie', 'Gentoo', 'Chinstrap']

pandas_dataset[label] = pandas_dataset[label].map(classes.index)

整数に変換した様子を見る．

In [ ]:
print(pandas_dataset[label])

## 訓練データと検証データに分ける  
  
344のデータの約1割を検証データに，残りを訓練データに分ける．ランダムに分けるので実行するたびに結果は異なる．

In [ ]:
np.random.seed(1)
# Use the ~10% of the examples as the testing set
# and the remaining ~90% of the examples as the training set.
test_indices = np.random.rand(len(pandas_dataset)) < 0.1
pandas_train_dataset = pandas_dataset[~test_indices]
pandas_test_dataset = pandas_dataset[test_indices]

print("Training examples: ", len(pandas_train_dataset))
# >> Training examples: 309

print("Testing examples: ", len(pandas_test_dataset))
# >> Testing examples: 35

## モデルの学習  
  
デフォルト値で行う．学習はすぐ終わる．

In [ ]:
tf_train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pandas_train_dataset, label=label)
model = tfdf.keras.CartModel()
model.fit(tf_train_dataset)

## モデルの評価
  
  まず決定木を表示する．見方は次の通り．  
* 左から右に見る
* 赤(0)がアデリー，青(1)がジェンツー，緑(2)がチンストラップ
* 分類された結果が棒グラフで表示されており，その上に条件が示されている．例えば最初の**flipper_length_mm >= 206.500**は**「羽の長さが206.5mm以上かどうか」**を意味する．Yesが上でNoが下．
* マウスを用いると，詳細な数字が表示される．

In [ ]:
tfdf.model_plotter.plot_model_in_colab(model, max_depth=10)

精度を計算する．訓練データ・検証データで共に97％程度となる．実際の精度は実行するたびにわずかに異なる(理由は考えること)．

In [ ]:
model.compile("accuracy")
print("Train evaluation: ", model.evaluate(tf_train_dataset, return_dict=True))
# >> Train evaluation:  {'loss': 0.0, 'accuracy': 0.96116}

tf_test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pandas_test_dataset, label=label)
print("Test evaluation: ", model.evaluate(tf_test_dataset, return_dict=True))
# >> Test evaluation:  {'loss': 0.0, 'accuracy': 0.97142}

## モデルの改良(Keras tunerの利用)

### Keras-tunerのインストール

In [ ]:
!pip install keras-tuner

### 最適化するパラメータの指定
  
次のパラメータをいくつかの候補を設けて検討する
* 決定木の深さ (min_examples)
* 検証データの割合 (validation_ratio)

In [ ]:
import keras_tuner as kt

def build_model(hp):
  model = tfdf.keras.CartModel(
      min_examples=hp.Choice("min_examples",
          # Try four possible values for "min_examples" hyperparameter.
          # min_examples=10 would limit the growth of the decision tree,
          # while min_examples=1 would lead to deeper decision trees.
         [1, 2, 5, 10]),
      validation_ratio=hp.Choice("validation_ratio",
         # Three possible values for the "validation_ratio" hyperparameter.
         [0.0, 0.05, 0.10]),
      )
  model.compile("accuracy")
  return model

tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    directory="/tmp/tuner",
    project_name="tune_cart")

tuner.search(x=tf_train_dataset, validation_data=tf_test_dataset)
best_model = tuner.get_best_models()[0]

print("Best hyperparameters: ", tuner.get_best_hyperparameters()[0].values)
# >> Best hyperparameters:  {'min_examples': 2, 'validation_ratio': 0.0}

### 改良したモデルの評価

残念ながら精度はさほど上がっていない．決定木は以前よりも深くなっている．

In [ ]:
model = tfdf.keras.CartModel(min_examples=2, validation_ratio=0.0)
model.fit(tf_train_dataset)

model.compile("accuracy")
print("Test evaluation: ", model.evaluate(tf_test_dataset, return_dict=True))
# >> Test evaluation:  {'loss': 0.0, 'accuracy': 1.0}

In [ ]:
tfdf.model_plotter.plot_model_in_colab(model, max_depth=10)

# ランダムフォレスト (random forest)
  
  [In Depth: Decision Trees and Random Forests](https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.08-Random-Forests.ipynb#scrollTo=hxFuYYC3G3eL)より  
  
  この例では決定木の例と異なり，scikit-learn(sklearn)を使用するので，コードは異なる．

## 決定木と過学習

分類用データを準備する．scikit-learnで4クラス・300個の二次元データを人工的に発生させる．

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=1.0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow');

決定木でモデルを作成する．

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X, y)

In [ ]:
def visualize_classifier(model, X, y, ax=None, cmap='rainbow'):
    ax = ax or plt.gca()

    # Plot the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=cmap,
               clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # fit the estimator
    model.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
                         np.linspace(*ylim, num=200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Create a color plot with the results
    n_classes = len(np.unique(y))
    contours = ax.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap=cmap, zorder=1)

    ax.set(xlim=xlim, ylim=ylim)

学習結果を見てみよう．

In [ ]:
visualize_classifier(DecisionTreeClassifier(), X, y)

おおむね正しく分類されているが，一部分で明らかな過学習(恣意的に見える分類区画)があるのが分かるだろうか．決定木は深くなれば深くなるほど精度は向上するが，このような過学習が発生する．

## ランダムフォレストの適用

ここでは「バギング」(bagging)という方法を用いる．バギングはデータの一部分を抽出して学習することを繰り返し(ランダムな決定木)，最後に平均化する方法である．  
次の例では決定木の数を100として，各々で全体の8割のデータをランダムに抽出する．

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8,
                        random_state=1)

bag.fit(X, y)
visualize_classifier(bag, X, y)

精度を求める．分類誤りは1個ぐらいのはず．

In [ ]:
z = bag.predict(X)
nerror = 0
for i in range(len(y)):
  if y[i] != z[i]:
    nerror += 1
print("accuracy={}".format(1-nerror/len(y)))